In [19]:
import pandas     
import numpy      
import datetime

In [20]:
pandas.options.display.max_rows = 10                                              
df = pandas.read_csv("stationnements.txt", sep='\t', header=0)   

In [21]:
def exploreData(data):
    if type(data) == pandas.core.frame.DataFrame:
        print("-"*60, "\n", data.head(), data.describe(), data.value_counts(),"\n", "-"*60)
    if type(data) == pandas.core.series.Series:
        print("-"*60, "\n", data.value_counts(), data.shape, "\n", "-"*60)    

In [22]:
# infos de df
# print(exploreData(df))

In [23]:
#On va voir les uniques valeurs dans chaque colonne et dans quelles colonnes se trouve NaN.
for col in df.columns.tolist():
    print("{} : {}".format(col, df[col].isnull().any()))

identifiant : False
Regime_prioritaire : False
Regime_particulier : False
Arrondissement : True
Zone_Residentielle : True
Tarification : True
Type_de_voie : True
Nom_de_la_voie : True
Parite : True
Longueur : False
Longueur_calculee : False
Signalisation_horizontale : False
Signalisation_verticale : False
Conformite_signalisation : False
Plage_horaire_1-Debut : True
Plage_horaire_1-Fin : True
Plage_horaire_2-Debut : True
Plage_horaire_2-Fin : True
Date_du_releve : False
Derniere_date_edition : True
Code_Voie_Ville_de_Paris : False
Numero_Sequentiel_Tronçon_Voie : True
Zone_ASP : True
Numero_Section_Territoriale_de_Voirie : True
geo_point_2d : False


In [24]:
# Q1 : Dans quel arrondissement il y a le plus de places disponibles aux plages de stationnement ouverts?
# (combien d'heures par arrondissement)

In [25]:
# Ségmentation : on tire les colonnes conernates
colonnes_Q1 = [ "identifiant", "Arrondissement", 
               "Plage_horaire_1-Debut", 
               "Plage_horaire_1-Fin", 
               "Plage_horaire_2-Debut", 
               "Plage_horaire_2-Fin"]
df_Q1 = df.loc[:, colonnes_Q1]
print(exploreData(df_Q1))

------------------------------------------------------------ 
   identifiant  Arrondissement Plage_horaire_1-Debut Plage_horaire_1-Fin  \
0        3210            13.0                 19:30               07:30   
1        3232            13.0                   NaN                 NaN   
2        3271             5.0                   NaN                 NaN   
3        3280             5.0                   NaN                 NaN   
4        3277             5.0                   NaN                 NaN   

  Plage_horaire_2-Debut Plage_horaire_2-Fin  
0                 09:30               16:30  
1                   NaN                 NaN  
2                   NaN                 NaN  
3                   NaN                 NaN  
4                   NaN                 NaN          Arrondissement
count    18858.000000
mean        12.963888
std          5.459199
min         -1.000000
25%          9.000000
50%         14.000000
75%         17.000000
max         22.000000 identifiant 

In [26]:
#=Commentaire=
#'value_counts' sur 'df_Q1' a sorti 646 lignes.
#On va voir les uniques valeurs dans chaque colonne et dans quelles colonnes se trouve NaN.

In [27]:
for col in df_Q1.columns.tolist():
    print(df_Q1[col].unique())

['3210' '3232' '3271' ... '10812' '10848' '10859']
[13.  5.  6. 11. 20. 18. 19. 15. 12.  4.  7. 16. 14.  9. 17. nan  2.  8.
 10. 21.  1. 22.  3. -1.]
['19:30' nan '09:30' '05:00' '07:00' '19:00' '07:30' '14:00' '19:20'
 '08:15']
['07:30' nan '16:30' '19:00' '14:30' '18:00' '07:00' '09:30' '17:30'
 '07:20' '20:00' '08:45']
['09:30' nan '19:30' '07:30' '05:00' '16:30' '20:00' '19:00' '09:38'
 '09:00' '16:15']
['16:30' nan '07:30' '14:30' '09:30' '19:30' '22:00' '18:30' '16:00'
 '16:29' '16:45' '17:30']


In [28]:
for col in df_Q1.columns.tolist():
    print(col, df_Q1[col].isnull().any())

identifiant False
Arrondissement True
Plage_horaire_1-Debut True
Plage_horaire_1-Fin True
Plage_horaire_2-Debut True
Plage_horaire_2-Fin True


In [29]:
# Toutes les colonnes 'plage horaire' ont le Nan,
# c'est-à-dire qu'il y a des places ont tous Nan dans les 'plage horaire'
# On compte 3 types de créneaux : 
#    A,Plage horaire 1; 
#    B,Plage horaire 2; 
#    C, 24h pour les place avec que Nan dans 'plage horaire'

In [30]:
# On va transférer la donné 'plage horaire' en minutes pour faciliter les calculs
# Changement des noms colonnes
df_Q1 = df_Q1.rename(columns={"Plage_horaire_1-Debut":"P1D", 
                                          "Plage_horaire_1-Fin":"P1F",
                                          "Plage_horaire_2-Debut":"P2D",
                                          "Plage_horaire_2-Fin":"P2F"})
print(df_Q1.columns)

Index(['identifiant', 'Arrondissement', 'P1D', 'P1F', 'P2D', 'P2F'], dtype='object')


In [31]:
# Remplacer Nan par '*'
df_Q1 = df_Q1.fillna(value=0)

In [32]:
for col in df_Q1.columns.tolist():
    print(col, df_Q1[col].isnull().any())

identifiant False
Arrondissement False
P1D False
P1F False
P2D False
P2F False


In [33]:
# 'plage horaire' en minutes
list_PH = ['P1D', 'P1F', 'P2D', 'P2F']

for row in range(df_Q1.shape[0]):
    tmp = 0
    for col in list_PH:
        if df_Q1[col][row] != 0:
            tmp = int(df_Q1[col][row][:2])*60 + int(df_Q1[col][row][-2:])            
            df_Q1[col][row] = tmp
        else:       
            pass
print('finit')

<ipython-input-33-f57300601961>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Q1[col][row] = tmp
finit


In [34]:
for col in df_Q1.columns.tolist():
    print(col, df_Q1[col].unique())

identifiant ['3210' '3232' '3271' ... '10812' '10848' '10859']
Arrondissement [13.  5.  6. 11. 20. 18. 19. 15. 12.  4.  7. 16. 14.  9. 17.  0.  2.  8.
 10. 21.  1. 22.  3. -1.]
P1D [1170 0 570 300 420 1140 450 840 1160 495]
P1F [450 0 990 1140 870 1080 420 570 1050 440 1200 525]
P2D [570 0 1170 450 300 990 1200 1140 578 540 975]
P2F [990 0 450 870 570 1170 1320 1110 960 989 1005 1050]


In [35]:
# Calculer les durées des plages
# Créer la colonne Duree et la remplir
df_Q1['Duree'] = pandas.Series([])

for row in range(df_Q1.shape[0]):
    # Plage Horaire 1
    créneau_1 = 0
    if df_Q1['P1F'][row] == 0:
        pass
    elif df_Q1['P1F'][row] > df_Q1['P1D'][row]:
        créneau_1 = df_Q1['P1F'][row] - df_Q1['P1D'][row]
    else:
        créneau_1 = 24*60 + (df_Q1['P1F'][row] - df_Q1['P1D'][row])

    # Plage Horaire 2
    créneau_2 = 0
    if df_Q1['P2F'][row] == 0:
        pass
    elif df_Q1['P2F'][row] > df_Q1['P2D'][row]:
        créneau_2 = df_Q1['P2F'][row] - df_Q1['P2D'][row]
    else:
        créneau_2 = 24*60 + (df_Q1['P2F'][row] - df_Q1['P2D'][row])
        
    # La colonne 'Duree'
    if (créneau_1 + créneau_2) == 0:
        df_Q1['Duree'][row] = 24*60
    else:
        df_Q1['Duree'][row] = créneau_1 + créneau_2
print(df_Q1['Duree'].value_counts(dropna=False))

<ipython-input-35-9873e9847e8f>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_Q1['Duree'] = pandas.Series([])
<ipython-input-35-9873e9847e8f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Q1['Duree'][row] = créneau_1 + créneau_2
<ipython-input-35-9873e9847e8f>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Q1['Duree'][row] = 24*60
1440.0    19760
1140.0      683
1110.0       10
1170.0        6
1740.0        6
          ...  
1139.0        1
1380.0        1
1132.0        1
210.0   

In [38]:
print(df_Q1.head())

  identifiant  Arrondissement   P1D  P1F  P2D  P2F   Duree
0        3210            13.0  1170  450  570  990  1140.0
1        3232            13.0     0    0    0    0  1440.0
2        3271             5.0     0    0    0    0  1440.0
3        3280             5.0     0    0    0    0  1440.0
4        3277             5.0     0    0    0    0  1440.0


In [40]:
# Enlever les colonnes 'Plage Horaire'
df_Q1_Clean = df_Q1.loc[:, ['identifiant', 'Arrondissement', 'Duree']]
df_Q1_Clean.head()

,identifiant,Arrondissement,Duree
0,3210,13.0,1140.0
1,3232,13.0,1440.0
2,3271,5.0,1440.0
3,3280,5.0,1440.0
4,3277,5.0,1440.0


In [42]:
df_Q1_Clean.to_csv('df_Q1_Clean', index=False)